In [ ]:
from pathlib import Path

In [ ]:
video_dir=Path("/data/data/MSRVTT_Zero_Shot_QA/videos/all")
gt_file_question=Path("/data/data/MSRVTT_Zero_Shot_QA/test_q1000.json")
gt_file_answers=Path("/data/data/MSRVTT_Zero_Shot_QA/test_a.json")

In [ ]:
import json

In [ ]:
with gt_file_question.open("r") as f:
    # print(json.load(f))
    pass

with gt_file_answers.open("r") as f:
    # print(json.load(f))
    pass

In [ ]:
import pandas as pd

In [ ]:
q_df = pd.read_json(gt_file_question)
a_df = pd.read_json(gt_file_answers)

df = pd.merge(q_df, a_df, on="question_id")
df

In [ ]:
sample = df.iloc[0]
sample

In [ ]:
Path(video_dir).joinpath(f"{sample.video_name}.mp4").exists()

In [ ]:
!ls /data/data/MSRVTT_Zero_Shot_QA/videos/all | grep 7121

In [ ]:
import requests

# URL of the API endpoint; adjust localhost and port if different
url = "http://localhost:5000/predict"

samples = df.iloc[:3]

data = {"inputs": []}

for _, sample in samples.iterrows():
    data["inputs"].append(
        {
            "video_path": Path(video_dir)
            .joinpath(f"{sample.video_name}.mp4")
            .resolve()
            .as_posix(),
            "text_prompt": sample.question,
        }
    )
data["temperature"] = 0.2
data["max_new_tokens"] = 1024
data

In [ ]:
# Send a POST request to the server
response = requests.post(url, json=data)

# Check if the request was successful
if response.status_code == 200:
    result = response.json()
    print("Predicted text:", result["predicted_texts"])
else:
    print("Failed to get response:", response.status_code, response.text)

In [ ]:
import concurrent.futures
import requests
import threading
from tqdm import tqdm

# Example batch pool
batch_pool = [
    data
]

# Model endpoints
model_endpoints = ["http://localhost:5000/predict"]

# Shared list to store results
results = []
results_lock = threading.Lock()

def worker(endpoint, batch_pool, pbar):
    while True:
        try:
            # Grab a batch from the pool
            batch = batch_pool.pop(0)
        except IndexError:
            # If the pool is empty, exit the loop
            break

        print(f"Submitting batch: {batch}")

        if response.status_code == 200:
            result = response.json()
            print("Predicted text:", result["predicted_texts"])
        else:
            print("Failed to get response:", response.status_code, response.text)

        print(result)

        # Append the result to the shared list
        with results_lock:
            results.append(result)

        # Update the progress bar
        pbar.update(1)

def process_batches(batch_pool, model_endpoints):
    total_batches = len(batch_pool)

    # Create a thread pool with the same number of workers as model endpoints
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(model_endpoints)) as executor:
        # Initialize the progress bar
        with tqdm(total=total_batches) as pbar:
            # Start a worker for each model endpoint
            futures = [executor.submit(worker, endpoint, batch_pool, pbar) for endpoint in model_endpoints]

            # Wait for all workers to finish
            concurrent.futures.wait(futures)

    # Print the results
    for result in results:
        print(result)


process_batches(batch_pool, model_endpoints)